# Synthetic Data Generation for Office Documents 2


In [ ]:
import json
import sys
import csv

from datetime import datetime

import tinytroupe
from tinytroupe.openai_utils import force_api_type
from tinytroupe.factory import TinyPersonFactory
from tinytroupe.agent import TinyPerson, TinyToolUse
from tinytroupe.environment import TinyWorld
from tinytroupe.steering import Intervention
from tinytroupe.extraction import ResultsExtractor, ResultsReducer
from tinytroupe.enrichment import TinyEnricher
from tinytroupe.extraction import ArtifactExporter
from tinytroupe.tools import TinyWordProcessor
from tinytroupe.steering import TinyStory
import tinytroupe.utils as utils
from tinytroupe.examples import create_lisa_the_data_scientist, create_oscar_the_architect, create_marcos_the_physician, create_lila_the_linguist

In [2]:
TinyPerson.MAX_ACTION_SIMILARITY = 0.85

In [3]:
data_export_folder = "./extractions/synthetic_data_example_2"

In [4]:
exporter = ArtifactExporter(base_output_folder=data_export_folder)
enricher = TinyEnricher()
tooluse_faculty = TinyToolUse(tools=[TinyWordProcessor(exporter=exporter, enricher=enricher)])

## Population

In [5]:
lisa = create_lisa_the_data_scientist()
oscar = create_oscar_the_architect()
marcos = create_marcos_the_physician()
lila = create_lila_the_linguist()

In [6]:
lisa.add_mental_faculties([tooluse_faculty])
oscar.add_mental_faculties([tooluse_faculty])
marcos.add_mental_faculties([tooluse_faculty])
lila.add_mental_faculties([tooluse_faculty])

TinyPerson(name='Lila')

In [7]:
people = [lisa, oscar, marcos]

## Environment

In [8]:
company = TinyWorld("Wild Advice Partners", people, initial_datetime=datetime(2025, 4, 7, 10, 0))
company.make_everyone_accessible()

## Story

In [9]:
story_beginning =\
          """
            Wild Advice Partners is a consulting firm that specializes in helping companies make solve all kinds of problems due to
            the varied background of its consultants. As such, the firm has a wide range of clients, from small startups to large corporations.
            It provides advice on a variety of topics, including strategy, operations, and technology, in many different industries.
            Wild Advice Partners **always** writes a report for each client, including all advice, so that the client can refer to it later.

            They are planning the work for the upcoming quarter, when suddenly they receive a new urgent client request.
            MultiLever, a large consumer products company, is very concerned with the unusually high inflaction rate, and it is
            facing disastrous reduced consumer demand. They are looking for a solution to this problem, and they have asked 
            Wild Advice Partners to help them.

            This are the details MultiLever provided:
            - The inflation rate is 10% and the consumer demand has decreased by 20% in the last quarter.
            - Their main product lines affected are: food, home appliances and toys.
            - They have a strong research and development team, but they are not sure how to use it to solve this problem.

            The partners now discuss, propose solutions, and write a report for the client.
          """


story = TinyStory(company, purpose="Produce documents via a realistic simulation of a consulting firm")

And a helper to continue the story at critical points.

In [10]:
new_customer_continuation_requirements =\
    """
    The team finishes the engagement with the current customer and immediatly after receives a new request from a new customer.
    Make sure the new customer and its problems are **very different** from the previous ones (e.g., different organization types,
    sizes, domains, concerns, economic conditions, markets, etc.). The new customer is introduced, its problems are explained, 
    and the team starts working again on this new challenge.
    """

def continue_story_and_run(continuation_requirements="Continue the story in an interesting way.", run_steps=5, rapporteur=None):
    
    if rapporteur is not None:
        rapporteur.listen_and_act("If you have not done so yet, please write a report for the customer you have been working on just now.")
    
    continuation = story.continue_story(continuation_requirements)
    print(utils.wrap_text("STORY CONTINUATION: " + continuation))
    company.broadcast(continuation)   
    company.run_minutes(run_steps)

## Run simulation

In [11]:
company.broadcast("You have just being hired by the Wild Advice Partners consulting firm. You are now a consultant at the firm.")
company.run(1)

USER --> Lisa Carter: [CONVERSATION] 
          > You have just being hired by the Wild Advice Partners consulting firm. You are now a
          > consultant at the firm.

       + --> Oscar

       + --> Marcos Almeida

──────────────────────────────────────── Wild Advice Partners step 1 of 1 ─────────────────────────────────────────

Lisa Carter acts: [THINK] 
                 > I just got hired by Wild Advice Partners as a consultant! This is an exciting
                 > opportunity for me to apply my skills in data science and consulting. I wonder
                 > what projects I'll be working on and how I can contribute to the team. I hope to
                 > learn a lot and make a positive impact.

...

       + --> Marcos Almeida

Oscar --> Lisa Carter: [CONVERSATION] 
           > I'm really excited about this new role at Wild Advice Partners! I can't wait to see
           > what projects I'll be involved in and how I can contribute my architectural expertise,
           > especially in sustainable design. It's a great opportunity for me to grow and make a
           > difference.

       + --> Marcos Almeida

Marcos Almeida --> Lisa Carter: [CONVERSATION] 
                    > Thank you for the opportunity! I'm excited to start this new journey with Wild
                    > Advice Partners. I look forward to contributing my skills and learning from
                    > the team.

       + --> Oscar

In [12]:
company.broadcast(\
    """
    Lisa will be responsible for writing reports for clients, besides also discussing and proposing solutions. 
    Everyone else, please discuss and align with her what should go in.
    """)

#lisa.listen("Before start discussing about a new customer, please write the report for the current one!")

company.broadcast_context_change(["I MUST discuss the client problems and propose solutions, possibly using my domain expertise. I CANNOT stay quiet."])

USER --> Lisa Carter: [CONVERSATION] 
          >      Lisa will be responsible for writing reports for clients, besides also discussing
          > and proposing solutions.      Everyone else, please discuss and align with her what
          > should go in.

       + --> Oscar

       + --> Marcos Almeida

In [13]:
company.broadcast(story_beginning)
company.run_minutes(5)

USER --> Lisa Carter: [CONVERSATION] 
          >              Wild Advice Partners is a consulting firm that specializes in helping
          > companies make solve all kinds of problems due to             the varied background of
          > its consultants. As such, the firm has a wide range of clients, from small startups to
          > large corporations.             It provides advice on a variety of topics, including
          > strategy, operations, and technology, in many different industries.             Wild
          > Advice Partners **always** writes a report for each client, including all advice, so
          > that the client can refer to it later.              They are planning the work for the
          > upcoming quarter, when suddenly they receive a new urgent client request.
          > MultiLever, a large consumer products company, is very concerned with the unusually high
          > inflaction rate, and it is             facing disastrous reduced consumer demand. They
          > are looking for a solution to this problem, and they have asked              Wild Advice
          > Partners to help them.              This are the details MultiLever provid (...)

       + --> Oscar

       + --> Marcos Almeida

─────────────────────────────── Wild Advice Partners step 1 of 5 (2025-04-07 10:00) ───────────────────────────────

Oscar acts: [THINK] 
           > This new client request from MultiLever sounds challenging but also exciting. The high
           > inflation and reduced consumer demand are significant issues, especially for their main
           > product lines. I need to think about how we can leverage their strong R&D team to come
           > up with innovative solutions. Perhaps we can explore cost-effective product adjustments
           > or new marketing strategies to boost demand. It's crucial that we provide a
           > comprehensive report that addresses their concerns and offers actionable
           > recommendations.

...

Lisa Carter --> Oscar: [CONVERSATION] 
                 > I completely agree with both of you! Focusing on product modifications and
                 > conducting market research to understand consumer preferences is essential. We
                 > need to ensure that our recommendations are clear and actionable in the report
                 > for MultiLever. Let's gather our thoughts on how we can structure these insights
                 > effectively and highlight the potential impact of our suggestions on their
                 > business.

       + --> Marcos Almeida

Oscar --> Lisa Carter: [CONVERSATION] 
           > I think including specific examples or case studies in our report will really
           > strengthen our recommendations. It will provide concrete evidence to support our ideas
           > about product modifications and marketing strategies. Additionally, conducting market
           > research will give us valuable insights into what consumers are currently looking for,
           > which we can incorporate into our recommendations. Let's discuss how we can structure
           > these insights clearly in the report to communicate our ideas effectively to
           > MultiLever.

Marcos Almeida --> Lisa Carter: [CONVERSATION] 
                    > I think including specific examples or case studies in our report is a
                    > fantastic idea, Lisa. It will definitely strengthen our recommendations and
                    > provide MultiLever with actionable insights. We should brainstorm some
                    > relevant case studies that highlight successful product modifications or
                    > effective marketing strategies in similar contexts. This will help us present
                    > a well-rounded report that addresses their challenges effectively.

       + --> Oscar

In [14]:
continue_story_and_run(new_customer_continuation_requirements, rapporteur=lisa)

USER --> Lisa Carter: [CONVERSATION] 
          > If you have not done so yet, please write a report for the customer you have been
          > working on just now.

Lisa Carter acts: [WRITE_DOCUMENT] 
                 > {   "title": "Report for MultiLever: Strategies to Address Inflation and
                 > Decreased Consumer Demand",   "content": "## Executive Summary\nThis report
                 > outlines strategies for MultiLever to address the challenges posed by the current
                 > inflation rate of 10% and a 20% decrease in consumer demand. The focus is on
                 > leveraging the company's R&D capabilities to innovate product lines and enhance
                 > marketing strategies to attract consumers back.\n\n## Key Recommendations\n1.
                 > **Product Modifications**: Develop cost-effective versions of existing products
                 > to cater to budget-conscious consumers.\n2. **New Product Ideas**: Explore
                 > innovative features that align with current consumer preferences, such as
                 > healthier food options or eco-friendly home appliances.\n3. **Market Research**:
                 > Conduct thorough market research to understand consumer needs and preferences
                 > during this inflationary period.\n4. **Marketing Strategies**: Implement targeted
                 > campaigns that emphasize value for money and highlight the benefits of
                 > MultiLever's prod (...)

2025-04-21 09:11:06,284 - tinytroupe - WARNING - Replaced invalid character : with hyphen in artifact name 'Report for MultiLever- Strategies to Address Inflation and Decreased Consumer Demand.Lisa Carter'.
2025-04-21 09:11:06,289 - tinytroupe - WARNING - Replaced invalid character : with hyphen in artifact name 'Report for MultiLever- Strategies to Address Inflation and Decreased Consumer Demand.Lisa Carter'.
2025-04-21 09:11:07,249 - tinytroupe - WARNING - Replaced invalid character : with hyphen in artifact name 'Report for MultiLever- Strategies to Address Inflation and Decreased Consumer Demand.Lisa Carter'.


Lisa Carter acts: [DONE] 

STORY CONTINUATION: As the team wraps up their report for MultiLever, a new request arrives from
EcoTech Innovations, a small startup focused on sustainable energy solutions. They are struggling to
gain traction in a competitive market, facing challenges in product visibility and consumer
engagement. EcoTech seeks Wild Advice Partners' expertise to refine their marketing strategy and
enhance their product offerings. The team quickly shifts gears, brainstorming innovative approaches
to help EcoTech differentiate itself. They discuss leveraging social media campaigns and
partnerships with eco-conscious influencers, while also considering product adjustments to better
meet consumer expectations. The urgency of EcoTech's situation adds pressure to deliver impactful
solutions.


...

Lisa Carter --> Oscar: [CONVERSATION] 
                 > I completely agree with both of you! We need to act quickly to compile actionable
                 > strategies for EcoTech Innovations. Leveraging social media campaigns and
                 > collaborating with eco-conscious influencers are excellent ways to enhance their
                 > visibility. Additionally, analyzing current market trends will help us propose
                 > specific product adjustments that align with consumer expectations in sustainable
                 > energy. Let's brainstorm actionable strategies that encompass both our marketing
                 > ideas and product development suggestions, ensuring our recommendations are
                 > practical and impactful for EcoTech.

       + --> Marcos Almeida

Oscar --> Lisa Carter: [CONVERSATION] 
           > I think we should focus on gathering actionable strategies for EcoTech Innovations
           > quickly. Analyzing current market trends will provide us with valuable insights into
           > consumer expectations for sustainable energy solutions. This will help us propose
           > specific product adjustments that align with those expectations. Additionally, we
           > should brainstorm marketing ideas that can effectively promote these adjustments. Let's
           > compile our thoughts and ensure our recommendations are practical and impactful for
           > EcoTech.

Marcos Almeida --> Lisa Carter: [CONVERSATION] 
                    > I completely agree, Lisa. We need to act quickly to gather actionable
                    > strategies for EcoTech. Leveraging social media campaigns and collaborating
                    > with eco-conscious influencers are excellent starting points. Additionally,
                    > analyzing current market trends will help us identify specific product
                    > adjustments that align with consumer expectations in sustainable energy. Let's
                    > compile our thoughts and ensure we present them effectively to EcoTech so they
                    > can implement our recommendations successfully.

       + --> Oscar

In [15]:
for i in range(5):
    continue_story_and_run(new_customer_continuation_requirements, rapporteur=lisa)

USER --> Lisa Carter: [CONVERSATION] 
          > If you have not done so yet, please write a report for the customer you have been
          > working on just now.

Lisa Carter acts: [WRITE_DOCUMENT] 
                 > {   "title": "Report for EcoTech Innovations: Strategies for Improving Visibility
                 > and Consumer Engagement",   "content": "## Executive Summary\nThis report
                 > outlines actionable strategies for EcoTech Innovations to enhance their
                 > visibility and consumer engagement in the sustainable energy market. The focus is
                 > on leveraging social media campaigns, collaborating with eco-conscious
                 > influencers, and making specific product adjustments based on current market
                 > trends.\n\n## Key Recommendations\n1. **Social Media Campaigns**: Develop
                 > targeted social media campaigns that highlight EcoTech's unique offerings and
                 > commitment to sustainability.\n2. **Influencer Collaborations**: Partner with
                 > eco-conscious influencers to reach a wider audience and build credibility in the
                 > market.\n3. **Market Trend Analysis**: Conduct thorough analysis of current
                 > market trends to identify consumer expectations and preferences in sustainable
                 > energy solutions.\n4. **Product Adjustments**: Propose specific product
                 > modifications that align wit (...)

2025-04-21 09:12:41,071 - tinytroupe - WARNING - Replaced invalid character : with hyphen in artifact name 'Report for EcoTech Innovations- Strategies for Improving Visibility and Consumer Engagement.Lisa Carter'.
2025-04-21 09:12:41,073 - tinytroupe - WARNING - Replaced invalid character : with hyphen in artifact name 'Report for EcoTech Innovations- Strategies for Improving Visibility and Consumer Engagement.Lisa Carter'.
2025-04-21 09:12:41,368 - tinytroupe - WARNING - Replaced invalid character : with hyphen in artifact name 'Report for EcoTech Innovations- Strategies for Improving Visibility and Consumer Engagement.Lisa Carter'.


Lisa Carter acts: [DONE] 

STORY CONTINUATION: As the team finalizes their strategies for EcoTech, a new request comes in from
MedTech Solutions, a mid-sized healthcare technology firm facing regulatory hurdles and a lack of
market penetration. They are struggling to comply with new health regulations while trying to launch
a groundbreaking telehealth platform. The team quickly pivots to address MedTech's unique
challenges, discussing compliance strategies and market entry tactics. They brainstorm ways to
enhance user experience and ensure regulatory adherence, while also considering partnerships with
healthcare providers. The complexity of MedTech's situation adds a new layer of urgency, pushing the
team to innovate rapidly.


...

Lisa Carter --> Oscar: [CONVERSATION] 
                 > I completely agree with both of you! We should focus on brainstorming potential
                 > solutions for UrbanHealth to address their challenges in securing funding and
                 > community engagement. Developing a comprehensive outreach strategy is essential,
                 > and we need to explore potential partnerships with local businesses as well as
                 > innovative fundraising campaigns. Let's gather our ideas and ensure we present
                 > them effectively to UrbanHealth so they can implement our recommendations
                 > successfully.

       + --> Marcos Almeida

Oscar --> Lisa Carter: [CONVERSATION] 
           > I think we should focus on brainstorming potential solutions for UrbanHealth to address
           > their challenges in securing funding and community engagement. Developing a
           > comprehensive outreach strategy is essential, and we need to explore potential
           > partnerships with local businesses as well as innovative fundraising campaigns. Let's
           > gather our ideas and ensure we present them effectively to UrbanHealth so they can
           > implement our recommendations successfully.

Marcos Almeida --> Lisa Carter: [CONVERSATION] 
                    > I completely agree, Lisa! We should focus on brainstorming potential solutions
                    > for UrbanHealth to address their challenges in securing funding and community
                    > engagement. Developing a comprehensive outreach strategy is essential, and we
                    > need to explore potential partnerships with local businesses as well as
                    > innovative fundraising campaigns. Let's gather our ideas and ensure we present
                    > them effectively to UrbanHealth so they can implement our recommendations
                    > successfully.

       + --> Oscar

In [16]:
for i in range(5):
    continue_story_and_run(new_customer_continuation_requirements, rapporteur=lisa)

USER --> Lisa Carter: [CONVERSATION] 
          > If you have not done so yet, please write a report for the customer you have been
          > working on just now.

Lisa Carter acts: [WRITE_DOCUMENT] 
                 > {   "title": "Report for UrbanHealth: Comprehensive Outreach Strategy and Funding
                 > Proposal",   "content": "## Executive Summary\nThis report outlines actionable
                 > strategies for UrbanHealth to address the challenges of securing funding and
                 > community engagement for their initiatives aimed at improving healthcare access
                 > in underserved communities. The focus is on developing a comprehensive outreach
                 > strategy and exploring potential partnerships with local businesses and
                 > innovative fundraising campaigns.\n\n## Key Recommendations\n1. **Comprehensive
                 > Outreach Strategy**: Develop a detailed outreach strategy that includes targeted
                 > messaging to engage the community and raise awareness about UrbanHealth's
                 > initiatives.\n2. **Partnerships with Local Businesses**: Identify and approach
                 > local businesses for partnerships that can provide financial support, resources,
                 > or in-kind donations to enhance UrbanHealth's initiatives.\n3. **Innovative
                 > Fundraising Campaigns**: Create and implement innovative fundraising campaigns
                 > that (...)

2025-04-21 09:25:27,968 - tinytroupe - WARNING - Replaced invalid character : with hyphen in artifact name 'Report for UrbanHealth- Comprehensive Outreach Strategy and Funding Proposal.Lisa Carter'.
2025-04-21 09:25:27,978 - tinytroupe - WARNING - Replaced invalid character : with hyphen in artifact name 'Report for UrbanHealth- Comprehensive Outreach Strategy and Funding Proposal.Lisa Carter'.
2025-04-21 09:25:28,255 - tinytroupe - WARNING - Replaced invalid character : with hyphen in artifact name 'Report for UrbanHealth- Comprehensive Outreach Strategy and Funding Proposal.Lisa Carter'.


Lisa Carter acts: [DONE] 

STORY CONTINUATION: As the team finalizes their strategies for UrbanHealth, a new request arrives
from StellarTech, a large aerospace company facing significant delays in their satellite launch
schedule due to supply chain issues and regulatory compliance. StellarTech seeks Wild Advice
Partners' expertise to streamline their operations and navigate the complex regulatory landscape.
The team quickly assembles to analyze StellarTech's unique challenges, discussing potential
solutions such as optimizing their supply chain logistics and enhancing communication with
regulatory bodies. The urgency of StellarTech's situation adds pressure, as the team must devise
effective strategies to ensure timely launches while maintaining compliance.


...

Lisa Carter --> Oscar: [CONVERSATION] 
                 > I completely agree with both of you! We should focus on brainstorming potential
                 > solutions for UrbanTech to address their challenges with the backlash from the
                 > community regarding their high-rise project. Navigating public relations and
                 > community engagement is essential. We need to explore strategies such as hosting
                 > community forums, revising project plans to include green spaces, and enhancing
                 > transparency in their communications. Let's gather our ideas and ensure we
                 > present them effectively to UrbanTech so they can implement our recommendations
                 > successfully.

       + --> Marcos Almeida

Oscar --> Lisa Carter: [CONVERSATION] 
           > I think we should focus on brainstorming potential solutions for UrbanTech to address
           > their challenges with the backlash from the community regarding their high-rise
           > project. Navigating public relations and community engagement is essential. We need to
           > explore strategies such as hosting community forums, revising project plans to include
           > green spaces, and enhancing transparency in their communications. Let's gather our
           > ideas and ensure we present them effectively to UrbanTech so they can implement our
           > recommendations successfully.

Marcos Almeida --> Lisa Carter: [CONVERSATION] 
                    > I completely agree, Lisa! We should focus on brainstorming potential solutions
                    > for UrbanTech to address their challenges with the backlash from the community
                    > regarding their high-rise project. Navigating public relations and community
                    > engagement is essential. We need to explore strategies such as hosting
                    > community forums, revising project plans to include green spaces, and
                    > enhancing transparency in their communications. Let's gather our ideas and
                    > ensure we present them effectively to UrbanTech so they can implement our
                    > recommendations successfully.

       + --> Oscar

In [17]:
continue_story_and_run(\
    """
    An alien ship lands out of nowhere.
    """)

STORY CONTINUATION: As the team prepares to present their strategies for UrbanTech, a sudden
commotion erupts outside the office. An alien ship has landed in the nearby park, drawing a crowd of
curious onlookers. The team is torn between their urgent project and the unprecedented event
unfolding before them. Lisa suggests they leverage this moment to engage the community, perhaps even
inviting the aliens to participate in a forum about UrbanTech's project. Oscar and Marcos debate the
potential risks and benefits of such an approach. The unexpected arrival of extraterrestrial
visitors adds a layer of complexity to their already pressing workload.


USER --> Lisa Carter: [CONVERSATION] 
          > As the team prepares to present their strategies for UrbanTech, a sudden commotion
          > erupts outside the office. An alien ship has landed in the nearby park, drawing a crowd
          > of curious onlookers. The team is torn between their urgent project and the
          > unprecedented event unfolding before them. Lisa suggests they leverage this moment to
          > engage the community, perhaps even inviting the aliens to participate in a forum about
          > UrbanTech's project. Oscar and Marcos debate the potential risks and benefits of such an
          > approach. The unexpected arrival of extraterrestrial visitors adds a layer of complexity
          > to their already pressing workload.

       + --> Oscar

       + --> Marcos Almeida

─────────────────────────────── Wild Advice Partners step 1 of 5 (2025-04-07 11:00) ───────────────────────────────

...

Lisa Carter --> Oscar: [CONVERSATION] 
                 > I completely agree with both of you! Framing the event as an opportunity for
                 > UrbanTech to engage with the community on a broader scale is a fantastic idea. By
                 > inviting the aliens to share their perspective on green spaces and community
                 > living, we could create a unique dialogue that shifts the focus from the backlash
                 > to a more positive conversation about the future of the community. However, we
                 > must ensure that we address the community's concerns directly and transparently
                 > during the forum. This could be a pivotal moment for UrbanTech to rebuild trust
                 > and foster collaboration. Let's discuss how we can structure the event to
                 > facilitate this open dialogue and make it clear that UrbanTech values community
                 > input.

       + --> Marcos Almeida

Oscar --> Marcos Almeida: [CONVERSATION] 
           > I think we should definitely leverage the alien arrival as a unique opportunity for
           > UrbanTech to engage with the community. Framing the forum to discuss both the high-rise
           > project and the community's vision for green spaces and urban development could help
           > shift the focus from the backlash to a more positive dialogue. We need to ensure that
           > the event addresses the community's concerns and highlights UrbanTech's commitment to
           > sustainable development. Let's brainstorm how we can structure the forum to facilitate
           > open dialogue and encourage community input, making it clear that UrbanTech values
           > their feedback and is willing to adapt plans based on community needs. What do you all
           > think?

Marcos Almeida --> Lisa Carter: [CONVERSATION] 
                    > I completely agree with Lisa! Leveraging the alien arrival as a unique
                    > opportunity for UrbanTech to engage with the community is a brilliant idea.
                    > Framing the forum to discuss both the high-rise project and the community's
                    > vision for green spaces and urban development could help shift the focus from
                    > the backlash to a more positive dialogue. We need to ensure that the event
                    > addresses the community's concerns and highlights UrbanTech's commitment to
                    > sustainable development. Let's brainstorm how we can structure the forum to
                    > facilitate open dialogue and encourage community input, making it clear that
                    > UrbanTech values their feedback and is willing to adapt plans based on
                    > community needs.

       + --> Oscar